In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import pandas as pd
import warnings
import numpy as np
from general_utils.constants import spectral_bands
warnings.filterwarnings("ignore")

In [ ]:
df_trees_raw = pd.read_csv("../../data/raw/raw_trainset.csv")
df1 = df_trees_raw.copy()
df1["time"] = pd.to_datetime(df1["time"])
df_aug = pd.read_csv("result.csv")
df2 = df_aug.copy()
df2 = df2.replace([np.inf, -np.inf], -999)
#df_aug["time"] = pd.to_datetime(df_aug["time"])

In [ ]:
df2

In [ ]:
common_cols = spectral_bands

In [ ]:
for col in common_cols:
    plt.figure(figsize=(6, 4))
    plt.hist(df1[col].dropna(), bins=30, alpha=0.5, label='Dataset 1')
    plt.hist(df2[col].dropna(), bins=30, alpha=0.5, label='Dataset 2')
    plt.title(f'Verteilung: {col}')
    plt.xlabel(col)
    plt.ylabel('Häufigkeit')
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
for col in common_cols:
    plt.figure(figsize=(4, 5))
    plt.boxplot([df1[col].dropna(), df2[col].dropna()], labels=['Data1', 'Data2'])
    plt.title(f'Boxplot: {col}')
    plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 🔹 Zeitspannen berechnen
def zeitspanne(df, name):
    agg = df.groupby('id')['time'].agg(['min', 'max'])
    agg.columns = ['start', 'ende']
    agg['dataset'] = name
    return agg

span1 = zeitspanne(df1, 'data1')
span2 = zeitspanne(df2, 'data2')

# 🔹 Alles zusammenführen
alle = pd.concat([span1, span2])

# -------------------------------------------------------------
# 🔹 1. Häufigkeit der Start- und Endzeitpunkte pro Tag

# Wir nehmen das Datum (ohne Uhrzeit)
alle['start'] = pd.to_datetime(alle['start'], errors='coerce')
alle['ende'] = pd.to_datetime(alle['ende'], errors='coerce')
alle['start_date'] = alle['start'].dt.date
alle['end_date'] = alle['ende'].dt.date

# Gruppieren nach Datum und zählen
starts = alle.groupby('start_date').size().reset_index(name='count_start')
ends = alle.groupby('end_date').size().reset_index(name='count_end')

# -------------------------------------------------------------
# 🔹 2. Plot: Häufigkeit über Datum
plt.figure(figsize=(10, 5))
plt.plot(starts['start_date'], starts['count_start'], label='Startzeitpunkte', color='tab:blue')
plt.plot(ends['end_date'], ends['count_end'], label='Endzeitpunkte', color='tab:orange')
plt.xlabel('Datum')
plt.ylabel('Anzahl IDs')
plt.title('Häufigkeit der Start- und Endzeitpunkte über die Zeit')
plt.legend()
plt.tight_layout()
plt.show()

# -------------------------------------------------------------
# 🔹 3. Optional: Vergleich pro Dataset (z. B. data1 vs data2)
for dataset in alle['dataset'].unique():
    subset = alle[alle['dataset'] == dataset]
    starts_ds = subset.groupby(subset['start'].dt.date).size().reset_index(name='count_start')
    ends_ds = subset.groupby(subset['ende'].dt.date).size().reset_index(name='count_end')
    
    plt.figure(figsize=(10, 5))
    plt.plot(starts_ds['start'], starts_ds['count_start'], label=f'{dataset} Starts')
    plt.plot(ends_ds['ende'], ends_ds['count_end'], label=f'{dataset} Enden')
    plt.xlabel('Datum')
    plt.ylabel('Anzahl IDs')
    plt.title(f'Häufigkeit der Start- und Endzeitpunkte – {dataset}')
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
span1.start.unique()

In [ ]:
span2.groupby("start").count()

In [ ]:
df2

In [ ]:
# Convert df2's id to string to match with df1
df2['id'] = df2['id'].astype(str)

# Get all unique ids from both dataframes
ids1 = set(df1['id'].astype(str))
ids2 = set(df2['id'])

# Find ids that are only in df1
unique_to_1 = ids1 - ids2

# Filter df1 to get only those rows
df_unique_to_1 = df1[df1['id'].astype(str).isin(unique_to_1)]

print(f"Number of unique rows in df1: {len(df_unique_to_1)}")
df_unique_to_1[['id']]